# Importações

In [ ]:
# Instala as dependências
!pip install -r ../../../requirements.txt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from math import sqrt

In [ ]:
# Evitando logs no plot de gráficos
pd.options.mode.chained_assignment = None

# Leitura do arquivo

In [ ]:
df = pd.read_csv('../../../data/tratado/dados_cvm_tratados.csv')

In [ ]:
# Coluna criada para futura comparação gráfica
df['Razao_Inadimplencia_Provisao'] = df['Inadimplencia_Total'] / df['Provisao_Total']

In [ ]:
df_para_treinamento = df.drop(["ID_Participante", "Data_Competencia"], axis=1) # para treinamento do modelo

# Contextualização
Tendo em vista o desafio de prever, através de um modelo de *machine learning*, as perdas financeiras e o mal provisionamento nos fundos de investimento em direitos creditórios (FIDCS), uma possível abordagem seria a identificação de fundos que compartilham de um comportamento semelhante por meio da clusterização. Um dos mais conhecidos algoritmos não supervisionados de agrupamento é o *K-Means*, que funciona de forma similar ao KNN e seu sistema de delimitação dos "nearest neighbours" ou vizinhos mais próximos.

Alguns dos pontos fortes desse algoritmo são sua eficiência computacional, seu caráter não supervisionado e sua simplicidade e facilidade de implementação e interpretação, motivos os quais tornaram indispensável a realização de um teste de comparação com o algoritmo. Entretanto, a simplicidade do algoritmo e sua dificuldade em detectar anomalias ou *outliers* podem ser um problema considerando os dados com os quais o grupo FIDCAS está trabalhando.

Concluímos anterior e provisoriamente que a clusterização não seria nosso objetivo central, dada a dificuldade de realizar agrupamentos com dados tão dispersos e distintos. Entretanto, com a nova abordagem de divisão dos fundos por carteira que foi adotada pelo grupo FIDCAS e com a necessidade de comparação de modelos para ratificar ou não a adesão definitiva ao modelo *Isolation Forest*, é preciso testar a possibilidade de clusterização dos fundos, que pode ser facilitada e mais efetiva devido à maior semelhança entre fundos de mesma carteira.

Logo, as seguintes seções desse notebook se destinam à modelagem com utilização do algoritmo *K-Means* das diferentes divisões de dados, feitas de acordo com o agrupamento das diferentes carteiras, e à avaliação dos resultados da mesma. Foram utilizadas as mesmas *features* aplicadas na implementação do *Isolation Forest*, pois ele parece atingir resultados satisfatórios com elas. Portanto, procuramos um modelo que supere seu desempenho utilizando características semelhantes.

# Separação de fundos por carteira

Utilizando o algoritmo de LabelEncoding, foi criada a coluna "Carteira_Classificacao_Encoded", que apresenta números que identificam fundos de diversas carteiras. Nesse contexto, as labels numéricas criadas são definidas da seguinte forma:

Setor Público = 0

Agronegócio = 1

Cartão = 2

Comercial = 3

Imobiliário = 4

Financeiro = 5

Industrial = 6

Factoring = 7

Multimercado = 8

Serviços = 9

Ações Judiciais = 10

In [ ]:
# Lista de carteiras com índices correspondentes à classificação
classificacao_encoding = [
    'SetorPublico',
    'Agronegocio',
    'Cartao',
    'Comercial',
    'Imobiliario',
    'Financeiro',
    'Industrial',
    'Factoring',
    'Multimercado',
    'Servicos',
    'AcoesJudiciais'
]

In [ ]:
# Separando os dados por carteira
dados_classificacao = {
    'Setor Público': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('SetorPublico')],
    'Agronegócio': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('Agronegocio')],
    'Cartão': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('Cartao')],
    'Comercial': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('Comercial')],
    'Imobiliário': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('Imobiliario')],
    'Financeiro': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('Financeiro')],
    'Industrial': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('Industrial')],
    'Factoring': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('Factoring')],
    'Multimercado': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('Multimercado')],
    'Serviços': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('Servicos')],
    'Ações Judiciais': df_para_treinamento.loc[df_para_treinamento['Carteira_Classificação_encoded'] == classificacao_encoding.index('AcoesJudiciais')]
}

In [ ]:
def calcula_distancias_k(dados : pd.DataFrame):
    '''
    Calcula e retorna a soma das distâncias quadradas retornadas
    pelo modelo treinado com diferentes números de clusters (1 a 11).

    ## Parâmetros
    dados (pd.Dataframe): O DataFrame a ser usado para cálculo das distâncias.

    ## Retorna
    Lista das distâncias relativas a cada número de clusters, sendo esses
    calculados em ordem crescente. 
    '''
    wcss = []

    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
        kmeans.fit(dados)
        wcss.append(kmeans.inertia_)

    return wcss

In [ ]:
def calcula_melhor_k(dados : pd.DataFrame, distancias : list = []):
    '''
    Calcula e retorna o melhor valor de K para treinamento do modelo baseado
    na soma das distâncias quadradas calculadas para cada número de clusters.

    ## Parâmetros
    dados (pd.Dataframe): O DataFrame a ser usado para cálculo das distâncias
    e do K.
    
    distancias (list): Caso já se tenha as distâncias calculadas em uma lista,
    é possível passar como parâmetro para não realizar o cálculo novamente.
    Necessário que a lista esteja ordenada de acordo com a ordem crescente de
    número de clusters.

    ## Retorna
    O melhor valor de K calculado.
    '''

    if len(distancias):
        wcss = distancias
    else:
        wcss = calcula_distancias_k(dados.drop(['Razao_Inadimplencia_Provisao'], axis=1))

    x1, y1 = 2, wcss[0]
    x2, y2 = len(wcss), wcss[-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 1

In [ ]:
def elbow_plot(dados : pd.DataFrame, classificacao : str):
    '''
    Constrói um gráfico no formato "Elbow Plot" para encontrar o valor ótimo
    de K para o treinamento do modelo *K-Means* e printa o esse valor de K.

    ## Parâmetros
    dados (pd.Dataframe): O DataFrame a ser usado para cálculo das distâncias
    e do K.
    
    classificacao (string): O tipo de carteira dos fundos sendo analizados
    (presentes no parâmetros "dados").

    ## Retorna
    None.
    '''

    distancias = calcula_distancias_k(dados)

    graph = plt.plot(range(1, 11), distancias, marker='o')
    plt.title(f'{classificacao}: Elbow Method')
    plt.xlabel('Número de clusters')
    plt.ylabel('Distâncias')
    plt.show()

    melhor_k = calcula_melhor_k(dados, distancias)
    print(f'O melhor valor de K para a carteira de {classificacao} é {melhor_k}\n')  

In [ ]:
# Plot de gráficos para cada divisão de dados por carteira
for classificacao in dados_classificacao.keys():
    elbow_plot(dados_classificacao[classificacao].drop('Razao_Inadimplencia_Provisao', axis=1), classificacao)

In [ ]:
def plota_grafico_dispersao(x : pd.Series, y : pd.Series, carteira : str):
    '''
    Constrói gráfico de dispersão relacionando duas colunas de um DataFrame.

    ## Parâmetros
    x (pd.Series): A coluna do eixo X do gráfico.
    
    y (pd.Series): A coluna do eixo Y do gráfico.
    
    carteira (string): O tipo de carteira dos fundos sendo analizados
    no gráfico. Opcional, apenas compõe o título do gráfico.

    ## Retorna
    None.
    '''
    graph = plt.scatter(x, y, c=x)
    if carteira:
        plt.title(f'{carteira}: {x.name} x {y.name}')
    else:
        plt.title(f'{x.name} x {y.name}')
    plt.show()

In [ ]:
def pipeline_kmeans(dados : pd.DataFrame, classificacao : str):
    '''
    Realiza todas as operações relativas ao treinamento dos modelos com o algoritmo *K-Means*.
    Constrói gráfico de dispersão relacionando os clusters e a inadimplência total.

    ## Parâmetros
    dados (pd.DataFrame): O DataFrame a ser usado para cálculo das distâncias
    e do K.
    
    classificacao (string): O tipo de carteira dos fundos sendo analizados
    (presentes no parâmetros "dados").

    ## Retorna
    None.
    '''
    k = calcula_melhor_k(dados)
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10)
    kmeans.fit_predict(dados.drop(['Razao_Inadimplencia_Provisao'], axis=1))
    dados['Cluster'] = kmeans.labels_

    plota_grafico_dispersao(dados['Cluster'], dados['Razao_Inadimplencia_Provisao'], classificacao)

In [ ]:
# Treino de modelo e plot de gráficos para cada divisão de dados por carteira
for classificacao in dados_classificacao.keys():
    pipeline_kmeans(dados_classificacao[classificacao], classificacao)

# Conclusão
O modelo desenvolvido com o algoritmo *K-Means* gerou resultados que diferem daqueles produzidos pelo *Isolation Forest* principalmente pela questão da clusterização realizada pelo primeiro em comparação à indicação do nível de anormalidade exposto pelo segundo. De forma geral, enquanto o *K-Means* agrupa dados que compartilham de características semelhantes em um K número de *clusters*, o *Isolation Forest* classifica os dados em anômalos (classificação "-1" do modelo) e normais (classificação "1" do modelo).

Devido à simplicidade do algoritmo *K-Means*, ele enfrenta dificuldades ao lidar com outliers e apresenta um desempenho não muito positivo ao trabalhar com *clusters* de formatos não convexos, ou seja, o modelo gerado não é apropriado para manusear dados com comportamentos tão distintos e sem padrão observável como os presentes no *dataset* do projeto. Ainda assim, para fins comparativos e de teste, o grupo realizou a modelagem para analisar os resultados gerados, buscando encontrar agrupamentos de fundos problemáticos nos diferentes tipos de carteira.

Com o desafio de comparar resultados de diferentes modelos não supervisionados e que ainda possuem processos tão diferentes de funcionamento em vista, criou-se a coluna "Razao_Inadimplencia_Provisao", uma medida que indica o quão maior que a provisão é a inadimplência (quanto maior o valor contido na coluna, maior a inadimplência em relação à provisão), dado que essas são duas *features* centrais e fundamentais para os objetivos do projeto e para análise de fundos problemáticos.

Além disso, também foi implementada a função "calcula_melhor_k", cujo objetivo é definido por encontrar o valor ótimo de K (número de *clusters* a serem gerados pelo modelo) para cada tipo diferente de carteira, a fim de maximizar a eficácia do algoritmo *K-Means*. Ela calcula esse valor encontrando o ponto no gráfico que mais se distancia da reta traçada entre o valor da soma das distâncias quadradas quando se treina o modelo com um *cluster* (em outras palavras, o primeiro ponto do gráfico de cotovelo, o qual possui o mais elevado valor de y) e o mesmo valor para o treinamento com onze *clusters* (máximo de *clusters* para testagem definido arbitrariamente; ponto com menor valor de y no gráfico de cotovelo). Esse ponto mais distante corresponde ao cotovelo do gráfico.

Com a construção de gráficos que demonstram a relação entre essa a razão entre a inadimplência e a provisão e os *clusters* gerados pelo algoritmo, é possível visualizar que os diferentes *clusters* contêm dados que compartilham certa semelhança no que tange essa característica da razão. Há *outliers* e eles tendem a estar mais presentes em um único *cluster*, porém, em todos os casos, esse mesmo cluster também apresenta uma grande quantidade de dados semelhantes aos presentes em outros agrupamentos. Ou seja, mesmo que fossem usados esses grupos de dados aparentemente anômalos, seria difícil filtrar o que realmente seria um fundo em risco, gerando um grande número de "falsos positivos".

Portanto, concluiu-se que o modelo não é o mais apropriado para construção da solução esperada no projeto quando comparado ao desenvolvido com o algoritmo *Isolation Forest*, tanto por sua dificuldade de classificar dados anômalos precisamente com o *dataset* utilizado quanto por trabalhar com clusterização e identificação de padrões, que não é a intenção principal do projeto considerando o caráter caótico dos dados. Ademais, sua simplicidade e consequentes limitações também prejudicam sua avaliação comparativa, haja vista a complexidade e os variados comportamentos dos dados com os quais o grupo FIDCAS está trabalhando.
